In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 17 10:11:23 2025

@author: randy

Semantic 


"""
# %pip install accelerate==1.3.0 #0.26.0
# %pip install sentence-transformers==3.4.1
# %pip install datasets==3.3.1

import datetime as dt
import pandas as pd
# import re
# import numpy as np
# import json
# import random
# import ast
# import copy
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pickle

#for vector embeddings
from sentence_transformers import SentenceTransformer, losses, InputExample, util

data_filename = "/home/randy/supportiv/mle_screening_dataset.csv"
org_huggingface_model_name = 'sentence-transformers/msmarco-MiniLM-L6-cos-v5'
embedding_model_filename = '/home/randy/supportiv/untrained_model'
nn_model_filename = "/home/randy/supportiv/untrained_nn_model.pkl"

In [ ]:
#############################################################################################################
# Step one: Build the search model <-------------------- you only need to do this one
#   1. load the data
#   2. load the model from HuggingFace and save locallly
#   3. encode all the answers
#   4. compute the nearest neighbor search model using the encoded answers
#   5. save the nearest neigbor model (as pickle file)

# read the raw data
train_data = pd.read_csv(data_filename)
train_data.columns
docs = train_data['answer'].astype(str).tolist()

#Load the pre-trained (not fine-tuned) embedding model
model = SentenceTransformer(org_huggingface_model_name)
model.save(embedding_model_filename) # save the embedding model locally

# encode the answeres with the embedding model
doc_emb = model.encode(docs)

# compute a nearest neighbor model with answer embeddings
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(doc_emb)

# same model to storage
pickle.dump(nbrs, open(nn_model_filename, 'wb'))

In [2]:
#############################################################################################################
# Step two: inference
#   1. load nearest neigbor model
#   2. load the HuggingFace model from local strage
#   3. load the orgnial answeres
#   4. ask a question
#   5. show the answers that are closet to the question

# load the nearest meighbor model
loaded_nn_model = pickle.load(open(nn_model_filename, 'rb'))

# load the embedding model
loaded_embedding_mode = SentenceTransformer(embedding_model_filename)

# load the orginal data set
train_data = pd.read_csv(data_filename)

/home/randy/anaconda3/envs/anapy310/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/randy/anaconda3/envs/anapy310/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:

def ask_question(the_question, num_results = 5):
    """Aks the semantic search a question and see the results"""
    new_embedding = loaded_embedding_mode.encode(the_question)
    distances, indices = loaded_nn_model.kneighbors([new_embedding], num_results)
    return_len = len(indices[0])

    return_results = []
    for i in range(return_len): #an_index in indices[0]:
        this_answer = train_data.loc[indices[0][i]]['answer']
        this_answers_distance = distances[0][i]
        return_results.append([i+1, the_question, this_answers_distance, this_answer])
        #print("******************************************************************************")
        #print(i+1, this_answers_distance, this_answer)
    
    return return_results

In [4]:
answers = ask_question("can someone be allergic to water?")

print(answers[0][1])

for i in range(len(answers)): #an_index in indices[0]:
    answer_rank = answers[i][0]
    this_answer = answers[i][3]
    answer_distance = answers[i][2]
    
    print("******************************************************************************")
    print(answer_rank, ":", answer_distance,":", this_answer)

can someone be allergic to water?
******************************************************************************
1 : 1.0669210209122924 : People drown when they get too much water in their lungs. You can drown in as little as an inch or two of water. Babies can drown in a sink or bathtub. Preschoolers are most likely to drown in a swimming pool. People who have seizure disorders are also at risk in the water. Drowning can happen quickly and silently.     Drowning precautions should include       -  Fences around pools    -  Supervising children near any body of water, including tubs    -  Not swimming or boating when under the influence of alcohol or sedatives    -  Wearing life jackets when boating    -  Learning CPR
******************************************************************************
2 : 1.1003814046830165 : Washing your hands frequently can help prevent upper respiratory infections, which can lead to an ear infection called otitis media. The ear infection otitis media can 

In [ ]:

answers = ask_question("tell me about glaucoma research?")

print(answers[0][1])

for i in range(len(answers)): #an_index in indices[0]:
    answer_rank = answers[i][0]
    this_answer = answers[i][3]
    answer_distance = answers[i][2]
    
    print("******************************************************************************")
    print(answer_rank, ":", answer_distance,":", this_answer)
